In [2]:
import notebook_common
import re
from importlib import reload
from collections import defaultdict

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l

)

p("Import common module")

Import common module


In [7]:
df = pd.read_csv(
    "d:/tmp/tiku_freq_raw.csv",
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
)


# \u2460-\u2473 ①-⑳（1-20 带圈数字）
# \uFF21-\uFF3A 全角的A-Z
# \uFF41-\uFF5A 全角的a-z
# \u0391-\u03A9  大写希腊字母
# \u03B1-\u03C9  小写希腊字母


pattern = r"[˚О○〇◯◎◇✖✕аА▽△▲♪☆★●|。∙｡･￥\"＂〞〝＇＊＃ⅭK₂ⅡⅢⅣ()\-－─−←（）〔〕｛｝〒〈〉／;；<=>@＜＞＠、､_【】％+＋！!&＆'%→？…·・“”」～~※℃\/「｢｣『』{}\u2460-\u2473――＝×÷（，＿:：,．\d\[\]\.\?\*a-zA-Z\uFF21-\uFF3A\uFF41-\uFF5A\u0391-\u03A9\u03B1-\u03C9]"

rec = re.compile(pattern)


# 使用 ~ 取反，保留不包含这些字符的行
filtered_df_freq = df[~df["spell"].str.contains(rec, na=False)].copy()
filtered_df_freq.sort_values(by="count", ascending=False, inplace=True)

total = filtered_df_freq['count'].sum()
filtered_df_freq['rate'] = filtered_df_freq['count'] / total


to_csv_sig(filtered_df_freq, "d:/tmp/filtered_df_freq.csv")


df_base = pd.read_csv(
    w_word_raw_merged_freq_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
)


df_t = pd.merge(
    df_base,
    filtered_df_freq,
    left_on="headword",
    right_on="spell",
    how="outer",
    indicator="check",
)

to_csv_sig(df_t, "d:/tmp/tiku_freq_raw_with_dfbase.csv")
l("ok")
df_t

# filtered_df

----------------------------------------ok----------------------------------------


,word_id,headword,frequency,spoken_freq,written_freq,fused_freq,pron_count,spell_type,Unnamed: 0,spell,count,rate,check
0,133627.0,110番,33628.0,0.0,108.0,795.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
1,168219.0,110番する,68220.0,0.0,7.0,52.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
2,103328.0,1つ,3329.0,0.0,4366.0,32158.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
3,180122.0,1つする,80123.0,0.0,2.0,15.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
4,134814.0,2ラン,34815.0,0.0,99.0,729.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107623,128590.0,ｔ,28591.0,1760.0,0.0,1174.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
107624,170832.0,ｔｈｅ,70833.0,60.0,0.0,40.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
107625,163306.0,ｖ,63307.0,119.0,0.0,79.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only
107626,182058.0,ｗ,82059.0,20.0,0.0,13.0,1.0,0.0,NaN,NaN,NaN,NaN,left_only


In [37]:
tiku_filtered2 = df_t[(df_t["spell_type"] == 1) & (df_t["check"] == "left_only")]


with open("D:/08.resource/01.gaokao/tiku/tiku_output.txt", "r", encoding="utf-8") as f:
    text = f.read()

words = tiku_filtered2["headword"].dropna().tolist()
# print(words)


word_counts = defaultdict(int)
for word in words:
    # 使用正则确保精确匹配（类似 grep -w）
    pattern = re.escape(word)
    count = len(re.findall(pattern, text))
    word_counts[word] = count


with open('D:/08.resource/01.gaokao/tiku/output_csv.csv', "w", encoding="utf-8-sig") as f:
    for word, count in word_counts.items():
        f.write(f"{word},{count}\n")

print(f"统计完成！结果保存到")


# with open(
#     "D:/08.resource/01.gaokao/tiku/output_csv.csv", "w", encoding="utf-8"
# ) as outfile:
#     for word in words:
#         if not word.strip():  # 跳过空词
#             continue
#         # 使用正则表达式统计词出现次数
#         count = len(re.findall(r"\b" + re.escape(str(word)) + r"\b", text, re.UNICODE))
#         outfile.write(f"{word},{count}\n")
l("ok")

统计完成！结果保存到
----------------------------------------ok----------------------------------------


In [10]:
df_ruku = pd.read_csv(
    "D:/08.resource/01.gaokao/tiku/ruku.csv",
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
)

df_ruku.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7464 entries, 0 to 7463
Data columns (total 69 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   阅读理解题目编号    6486 non-null   float64
 1   文章正文        6486 non-null   object 
 2   第1小题题目      6486 non-null   object 
 3   A选项         6485 non-null   object 
 4   选项          6485 non-null   object 
 5   C选项         6482 non-null   object 
 6   D选项         6381 non-null   object 
 7   第2小题题目      6401 non-null   object 
 8   A选项.1       6399 non-null   object 
 9   选项.1        6399 non-null   object 
 10  C选项.1       6396 non-null   object 
 11  D选项.1       6297 non-null   object 
 12  第3小题题目      6280 non-null   object 
 13  A选项.2       6280 non-null   object 
 14  选项.2        6280 non-null   object 
 15  C选项.2       6277 non-null   object 
 16  D选项.2       6179 non-null   object 
 17  第4小题题目      6101 non-null   object 
 18  A选项.3       6101 non-null   object 
 19  选项.3        6101 non-null  